In [11]:
import pandas as pd
import numpy as np

file_path = '/Users/yuanpingzhou/project/workspace/python/kaggle/Zillow/data/p1/train.hdf'
df_train = pd.read_hdf(path_or_buf= file_path,key = 'test')

In [12]:
# missing_df = df_train.isnull().sum(axis=0).reset_index()
# missing_df.columns = ['column_name', 'missing_count']
# missing_df[missing_df['missing_count'] > 0]

In [13]:
for col,dt in zip(df_train.columns,df_train.dtypes):
    if(dt == np.float64):
        print(col,dt)
        df_train[col] = df_train[col].astype(np.float32)
    elif(dt == np.int64):
        print(col,dt)
        df_train[col] = df_train[col].astype(np.int32)
df_train.dtypes.value_counts()

parcelid int64
logerror float64
lastgap float64
monthyear int64


int8       1604
float32      32
int32         2
dtype: int64

In [14]:
print(len(df_train))
df_train = df_train[(df_train['logerror'] > -0.4) & (df_train['logerror'] < 0.42)]
print(len(df_train))

x_train = df_train.drop(['logerror','parcelid'],axis= 1)
y_train = df_train['logerror']
l_train_columns = x_train.columns

# split = 80000
# x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]
x_train = x_train.values.astype(np.float32, copy=False)
# x_valid = x_valid.values.astype(np.float32, copy=False)


90275
88528


In [15]:
import gc
import lightgbm

d_train = lightgbm.Dataset(x_train, label=y_train)
# d_valid = lightgbm.Dataset(x_valid, label=y_valid)

params = {}
params['max_bin'] = 8
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'mae'
params['sub_feature'] = 0.8
params['bagging_fraction'] = 0.85  # sub_row
params['num_leaves'] = 128
params['min_data'] = 300
params['min_hessian'] = 0.01

#l_learning_rate = [0.001*math.pow(2.0,i) for i in range(7)]
l_learning_rate = [0.018 + 0.002*i for i in range(5)]
l_bagging_freq = [20+i*10 for i in range(5)]
# l_learning_rate = [0.01]
# l_bagging_freq = [40]
        
BestParams = {'learning_rate':0.0,'bagging_freq':0}
BestMAE = 1.0
for lr in l_learning_rate:
    for bf in l_bagging_freq:
        params['learning_rate'] = lr
        params['bagging_freq'] = bf
        
        model = lightgbm.cv(params, d_train, 100, nfold=5,verbose_eval= True)
        if(model.get('l1-mean')[-1] < BestMAE):
            BestMAE = model.get('l1-mean')[-1]
            BestParams['learning_rate'] = lr
            BestParams['bagging_freq'] = bf
print(BestParams)


[1]	cv_agg's l1: 0.0532862 + 0.000349121
[2]	cv_agg's l1: 0.0532603 + 0.000350263
[3]	cv_agg's l1: 0.0532364 + 0.000350379
[4]	cv_agg's l1: 0.0532137 + 0.000350561
[5]	cv_agg's l1: 0.0531917 + 0.000351656
[6]	cv_agg's l1: 0.0531702 + 0.000352194
[7]	cv_agg's l1: 0.0531557 + 0.000352419
[8]	cv_agg's l1: 0.0531359 + 0.000353492
[9]	cv_agg's l1: 0.0531164 + 0.000353848
[10]	cv_agg's l1: 0.0530979 + 0.00035303
[11]	cv_agg's l1: 0.0530861 + 0.000353236
[12]	cv_agg's l1: 0.0530686 + 0.000352999
[13]	cv_agg's l1: 0.0530569 + 0.000352415
[14]	cv_agg's l1: 0.0530396 + 0.000352178
[15]	cv_agg's l1: 0.0530227 + 0.000353497
[16]	cv_agg's l1: 0.0530115 + 0.000353189
[17]	cv_agg's l1: 0.0529969 + 0.000353456
[18]	cv_agg's l1: 0.052982 + 0.000352934
[19]	cv_agg's l1: 0.0529685 + 0.000352527
[20]	cv_agg's l1: 0.0529593 + 0.000351896
[21]	cv_agg's l1: 0.0529487 + 0.000351274
[22]	cv_agg's l1: 0.0529359 + 0.000352568
[23]	cv_agg's l1: 0.0529236 + 0.000353966
[24]	cv_agg's l1: 0.0529116 + 0.000354237
[25

In [6]:

params['learning_rate'] = 0.02
params['bagging_freq'] = 30
x_train = df_train.drop(['logerror','parcelid'],axis= 1)
y_train = df_train['logerror']
d_train = lightgbm.Dataset(x_train, label=y_train)
model = lightgbm.train(params,d_train,100,verbose_eval= True)


In [7]:
file_path = '/Users/yuanpingzhou/project/workspace/python/kaggle/Zillow/data/p1/test.hdf'
df_test = pd.read_hdf(path_or_buf= file_path,key = 'test')
sub = pd.DataFrame(index = df_test.index)
sub['ParcelId'] = df_test['parcelid']

# df_test.drop(l_test_transdate,axis = 1,inplace=True)
# df_test.drop('parcelid',axis = 1,inplace=True)

In [10]:
import time

N = 200000
l_test_transdate = ['201610', '201611', '201612', '201710', '201711', '201712']
start = time.time()
for d in l_test_transdate:
    s0 = time.time()
    print("Start prediction ...")
    
    l_test_columns = ['%s%s' % (c,d) if (c in ['lastgap','monthyear','buildingage']) else c for c in l_train_columns]
    x_test = df_test[l_test_columns]
    for idx in range(0,len(x_test),N):
        x_test_block = x_test[idx:idx + N].values.astype(np.float32, copy=False)
        model.reset_parameter({"num_threads":4})
        ret = model.predict(x_test_block)
        sub.loc[x_test[idx:idx + N].index,d] = ret
        print(np.mean(np.abs(ret)))

    e0 = time.time()
    print('%s done. time elapsed %ds' % (d,(e0 - s0)))
end = time.time()
sub.to_csv('lgb_starter.csv', index=False, float_format='%.4f')
print('time elapsed %ds' % (end - start))

Start prediction ...
0.0134480309018
0.013446911263
0.0134523743616
0.0134417438461
0.013470370291
0.0134561359555
0.0134484411045
0.0134581279229
0.0134458168905
0.0134590822073
0.0134812310434
0.0134442905426
0.0134446496209
0.0134536666461
0.0134716707567
201610 done. time elapsed 152s
Start prediction ...
0.0135621312283
0.0135625854368
0.0135678780184
0.0135567867748
0.0135852429481
0.013571600723
0.0135637050965
0.0135738054457
0.0135615245209
0.0135744162027
0.0135970168175
0.0135599204942
0.0135598993446
0.0135686214633
0.013587367122
201611 done. time elapsed 157s
Start prediction ...
0.0135621312283
0.0135625854368
0.0135678780184
0.0135567867748
0.0135852429481
0.013571600723
0.0135637050965
0.0135738054457
0.0135615245209
0.0135744162027
0.0135970168175
0.0135599204942
0.0135598993446
0.0135686214633
0.013587367122
201612 done. time elapsed 153s
Start prediction ...
0.013464057846
0.0134608418982
0.0134681916956
0.0134565671125
0.0134851496472
0.0134700203051
0.013463280967